In [ ]:
#!pip install pyg_lib torch_scatter torch_sparse torch_cluster torch_spline_conv torch_geometric -f https://data.pyg.org/whl/torch-1.13.0+cpu.html

## OGB(L) - BIOKG

[Open Graph Benchmark: Datasets for Machine Learning on Graphs](https://arxiv.org/abs/2005.00687) is one of the most used benchmark in machine learning on graphs. OGB contains graph datasets that are managed by data loaders. The loaders handle downloading and pre-processing of the datasets. Additionally, OGB has standardized evaluators and leaderboards to keep track of state-of-the-art results.

**Dataset description**: For this lecture, we will focus on OGBL - BIOKG, a knowledge graph related to bio-medical field available in OGB to perform multi-relational link prediction tasks.  The ogbl-biokg dataset is a Knowledge Graph (KG), which it was created using data from a large number of biomedical data repositories. It contains 5 types of entities: diseases (10,687 nodes), proteins (17,499), drugs (10,533 nodes), side effects (9,969 nodes), and protein functions (45,085 nodes). There are 51 types of directed relations connecting two types of entities, including 38 kinds of drug-drug interactions, 8 kinds of protein-protein interaction, as well as drug-protein, drug-side effect, function-function relations. All relations are modeled as directed edges, among which the relations connecting the same entity types (e.g., protein-protein, drug-drug, function-function) are always symmetric, i.e., the edges are bi-directional.

**Prediction task**: Prediction of drugs' side effect (i.e. link prediction between drug and side effect entities).

**Dataset splitting**: They adopt random split. While splitting the triplets according to time is an attractive alternative, it is incredibly challenging to obtain accurate information as to when individual experiments and observations underlying the triplets were made.

In [ ]:
#Retrive BioKG from OGB
from ogb.linkproppred import PygLinkPropPredDataset

dataset = PygLinkPropPredDataset(name = 'ogbl-biokg') 

graph = dataset[0]

In [ ]:
#Construct an HeteroData object from OGB-BIOKG
from torch_geometric.data import HeteroData
import torch

data = HeteroData()

for tnode, num in graph.num_nodes_dict.items():
    data[tnode].x = torch.Tensor([[1] for i in range(num)])
    
for tedge, edge_index in graph.edge_index_dict.items():
    data[tedge].edge_index = edge_index

In [ ]:
data['drug']
#E.g.: there are 10687 disease with 1 node feature. 
#There are 73547 edge between disease and protein

In [ ]:
data['drug'].x

In [ ]:
data['drug', 'drug-sideeffect', 'sideeffect'].edge_index

In [ ]:
data.x_dict

In [ ]:
data.edge_index_dict

In [ ]:
data.metadata() #in position 0 you have the list of node types, in pos 1 the list of edge types

## MetaPath2Vec

We can refer to the [example](https://github.com/pyg-team/pytorch_geometric/blob/master/examples/hetero/metapath2vec.py) available in PyG

## Factorization Methods

From March 2023, factorization methods like DistMult, TransE, and ComplEx are available in [PyG](https://pytorch-geometric.readthedocs.io/en/latest/modules/nn.html#kge-models)! 

We also suggest the implementations available in [libKGE](https://github.com/uma-pi1/kge)

## HeteroGNN Models

### RGCN

[RGCNConv](https://pytorch-geometric.readthedocs.io/en/latest/generated/torch_geometric.nn.conv.RGCNConv.html#torch_geometric.nn.conv.RGCNConv) is implemented and available in PyG. However, it does not support HeteroData but just Data with an attribute on the edge types. For this reason, we chose to present HeteroConv, a generic wrap for hetero graph learning that could be useful for any kind of HeteroGNN-based architecture. In the following architecture, we will use [GraphSAGE](https://pytorch-geometric.readthedocs.io/en/latest/generated/torch_geometric.nn.conv.SAGEConv.html) instead of [GCN](https://pytorch-geometric.readthedocs.io/en/latest/generated/torch_geometric.nn.conv.GCNConv.html#torch_geometric.nn.conv.GCNConv) due to an heterogeneity-unaware implementation problem.

In [ ]:
from torch_geometric.nn import HeteroConv, Linear, SAGEConv
import torch.nn.functional as F

In [ ]:
class BioRGCN(torch.nn.Module):
    
    def __init__(self, metadata):
        super().__init__()
        torch.manual_seed(1234567) #manual seed for reproducibility
        edge_types = metadata[1]
        self.conv1 = HeteroConv({edge_t: SAGEConv((1,1), 32, add_self_loops=False) for edge_t in edge_types}, aggr='sum')
        self.conv2 = HeteroConv({edge_t: SAGEConv((32,32), 2, add_self_loops=False) for edge_t in edge_types}, aggr='sum')
        
    def reset_parameters(self):
        self.conv1.reset_parameters()
        self.conv2.reset_parameters()

    def forward(self, x_dict, edge_index_dict, edge_label_index):
        #x : node feature matrix, edge_index : structure of the graph
        
        out_dict = self.conv1(x_dict, edge_index_dict)
        out_dict = {node: out.relu() for node,out in out_dict.items()}
        
        out_dict = self.conv2(out_dict, edge_index_dict)
        
        #link prediction decoder on ('drug', 'drug-sideeffect', 'sideeffect')
        h_src = out_dict['drug'][edge_label_index[0]]
        h_dst = out_dict['sideeffect'][edge_label_index[1]]
        h_sim = h_src * h_dst
        h = torch.sum(h_sim,dim=1)
        
        return h

In [ ]:
model = BioRGCN(data.metadata())
model.reset_parameters()

In [ ]:
model

![How Hetero graph learning works](heteroconv.PNG)

### Link Prediction

#### Link split and negative sampling

In [ ]:
#Perform random link split
from torch_geometric.transforms import RandomLinkSplit

link_split = RandomLinkSplit(num_val=0.0,num_test=0.25, edge_types = data.metadata()[1])
train_link, val_link, test_link = link_split(data)

In [ ]:
train_link

In [ ]:
# edge_label : 1 for closed link, 0 for open link
# edge_label_index: edge_index + negative_sampling edge_index

`RandomLinkSplit` performs split + negative sampling: this is perfect for static networks. Suppose you want to test your model in a future time interval... you do not need to perform a link split. How to perform negative sampling?

#### Training

In [ ]:
from sklearn.metrics import roc_auc_score

In [ ]:
model = BioRGCN(data.metadata())
model.reset_parameters()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
criterion =  torch.nn.BCEWithLogitsLoss() #change loss function

def train_linkpre():
    model.train()
    optimizer.zero_grad()  # Clear gradients.
    pred = model(train_link.x_dict, train_link.edge_index_dict,\
                train_link['drug', 'drug-sideeffect', 'sideeffect'].edge_label_index)
    # Perform a single forward pass.
    
    loss = criterion(pred, train_link['drug', 'drug-sideeffect', 'sideeffect'].edge_label.type_as(pred))  # Compute the loss solely based on the training nodes.
    loss.backward()  # Derive gradients.
    optimizer.step()  # Update parameters based on gradients.
    return loss

for epoch in range(1, 101):
    loss = train_linkpre()
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}')

In [ ]:
def test_linkpre(test_link):
    model.eval()
    out = model(test_link.x_dict, test_link.edge_index_dict,\
               test_link['drug', 'drug-sideeffect', 'sideeffect'].edge_label_index)
    
    pred_cont = torch.sigmoid(out).cpu().detach().numpy()
    
    # EVALUATION
    test_label = test_link['drug', 'drug-sideeffect', 'sideeffect'].edge_label.cpu().detach().numpy() #retrieve test set labels
    test_roc_score = roc_auc_score(test_label, pred_cont) #comput AUROC score for test set
    
    return test_roc_score

In [ ]:
roc_train = test_linkpre(train_link)
roc_test = test_linkpre(test_link)
print(f'Train AUROC: {roc_train:.4f}\nTest AUROC: {roc_test:.4f}')

### HAN

Beside HeteroConv wrapper, there are some graph convolutional operator natively developed for heterogenous graph learning. [HANConv](https://pytorch-geometric.readthedocs.io/en/latest/generated/torch_geometric.nn.conv.HANConv.html#torch_geometric.nn.conv.HANConv) is one of them.

In [ ]:
from torch_geometric.nn import HANConv

class BioHAN(torch.nn.Module):
    
    def __init__(self, metadata):
        super().__init__()
        torch.manual_seed(1234567) #manual seed for reproducibility
        self.conv1 = HANConv(1, 32, metadata=metadata)
        self.conv2 = HANConv(32, 2, metadata=metadata)
        
    def reset_parameters(self):
        self.conv1.reset_parameters()
        self.conv2.reset_parameters()

    def forward(self, x_dict, edge_index_dict, edge_label_index):
        #x : node feature matrix, edge_index : structure of the graph
        
        out_dict = self.conv1(x_dict, edge_index_dict)
        out_dict = {node: out.relu() for node,out in out_dict.items()}
        
        out_dict = self.conv2(out_dict, edge_index_dict)
        
        #link prediction decoder on ('drug', 'drug-sideeffect', 'sideeffect')
        h_src = out_dict['drug'][edge_label_index[0]]
        h_dst = out_dict['sideeffect'][edge_label_index[1]]
        h_sim = h_src * h_dst
        h = torch.sum(h_sim,dim=1)
        
        return h

In [ ]:
model = BioHAN(data.metadata())
model.reset_parameters()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
criterion =  torch.nn.BCEWithLogitsLoss() #change loss function

In [ ]:
for epoch in range(1, 101):
    loss = train_linkpre()
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}')

In [ ]:
roc_train = test_linkpre(train_link)
roc_test = test_linkpre(test_link)
print(f'Train AUROC: {roc_train:.4f}\nTest AUROC: {roc_test:.4f}')

### Exercise 0 - HeteroConv + GAT = HAN?

Can you implement HAN using just HeteroConv wrapper with GATConv as operator for each layer ? Why? Answer this question without implementing the architecture. Then, investigate how you can implement HAN using HeteroConv.

### Exercise 1 - Node feature and expressiveness

In this notebook, we have a BioKG dataset without node feature.  
We used a constant encoder to initialize node features. However, this kind of initialization limit the [expressive power of GNN](https://arxiv.org/abs/2112.09992) to 1-WL test for graph isomorphism (more precisely, to [1-RWL](https://arxiv.org/abs/2211.17113)). Change the node feature inizialization to be more expressive. 

For instance, consider these 3 following options:
- Compute the [Local Degree Profile](https://pytorch-geometric.readthedocs.io/en/latest/generated/torch_geometric.transforms.LocalDegreeProfile.html#torch_geometric.transforms.LocalDegreeProfile).
- Count graphlets (e.g. triangles) in which the nodes are involved.
- Compute some well-know structural indices and centrality measures like PageRank, degree, average clustering coefficient. Networkx provides useful functions to compute structural features.



### Exercise 2 - Mean Reciprocal Rank

Beside AUPRC, a common evaluation metric used for link prediction in KG is the Mean Reciprocal Rank (MRR). 
Given a graph $G = (N,E)$, the computation behind MRR can be summarized in the following steps:
- Compute the prediction score of a real edge $(i,j) \in E$. (Standard forward GNN computation)
- Given a real edge $(i,j)$ in the dataset, corrupt it $n$ times obtaining no existing edges $(i,k), k \in N$. Be aware to not leak real edges from train to test set and viceversa(i.e. perform this step on the whole dataset before the train-test split).
- Compute the prediction scores for corrupted edges.
- Sort the corrupted edges plus the real edge by non increasing prediction score.
- Compute the rank $r$ of the real edge in the sorted list.
- Compute the reciprocal rank $\frac{1}{r}$ of the real edge.
- Repeat the previous steps for each real edge in the dataset.
- Average the reciprocal ranks.

Unfortunately, mean reciprocal rank on link prediction task is not available in PyG. Write a function to evaluate the prediction performance of BioRGCN with MRR. To corrupt the edges, you can use the function [structured_negative_sampling](https://pytorch-geometric.readthedocs.io/en/latest/modules/utils.html#torch_geometric.utils.structured_negative_sampling). It is sufficient to corrupt the edges one time.

### Exercise 3 - DistMult

Consider the problem of predicting protein-protein interactions (PPIs) in the given Biomedical KG. We can observe 8 different kinds of PPIs in the dataset. If we use the dot product as decoder, given two entities we obtain the same prediction score for each type of interaction. 

How we can distinguish the prediction scores of different interactions? Consider heterogeneity-aware decoders inspired by tensor decomposition techniques on KGs. One of the most famous tensor decomposition model for Knowledge Base Completion is [DistMult](https://arxiv.org/abs/1412.6575).

Develop a DistMult decoder for PPI predictions. Consider the suggestions by Mathias Fey on a [Github thread](https://github.com/pyg-team/pytorch_geometric/discussions/2513) about DistMult on the PyG repository to implement it. 

Train a HeteroGNN model using the OGB-BIOKG dataset to solve the PPI prediction task. Evaluate the performance over all the PPIs (for instance, by averaging them).